# Custom User Interface (ui.json)


This section provides training on how to create custom user interface within Geoscience ANALYST using the [ui.json
standard](https://geoh5py.readthedocs.io/en/stable/content/uijson_format/index.html).

## UI.json format

# Example 1: Electric field calculator (revisited)

 Copyright (c) 2022 Mira Geoscience Ltd.